In [1]:
## libraries
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
import os, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import time

In [2]:
## dataset
def load_cifar10(path):
    data, labels = [], []
    for i in range(1, 6):
        with open(os.path.join(path, f"data_batch_{i}"), 'rb') as f:
            batch = pickle.load(f, encoding='bytes')
            data.append(batch[b'data'])
            labels += batch[b'labels']
    X = np.concatenate(data).astype(np.float32) / 255.0
    y = np.array(labels).reshape(-1, 1)
    return X, y

def load_cifar10_test(path):
    with open(os.path.join(path, "test_batch"), 'rb') as f:
        batch = pickle.load(f, encoding='bytes')
        X = batch[b'data'].astype(np.float32) / 255.0
        y = np.array(batch[b'labels']).reshape(-1, 1)
    return X, y

In [5]:
## ANN 
class ANN:
    def __init__(self, input_dim, hidden_layers, output_dim, lr=0.1):
        self.lr = lr
        self.layers = []
        prev = input_dim
        for h in hidden_layers:
            self.layers.append({
                "W": np.random.randn(prev, h) * 0.01,
                "b": np.zeros((1, h))
            })
            prev = h
        self.out = {
            "W": np.random.randn(prev, output_dim) * 0.01,
            "b": np.zeros((1, output_dim))
        }

    def sigmoid(self, z): 
        return 1 / (1 + np.exp(-z))

    def sigmoid_deriv(self, a): 
        return a * (1 - a)

    def softmax(self, z):
        z -= np.max(z, axis=1, keepdims=True)
        exp = np.exp(z)
        return exp / np.sum(exp, axis=1, keepdims=True)

    def forward(self, X):
        self.cache = [{'a': X}]
        a = X
        for layer in self.layers:
            z = a @ layer['W'] + layer['b']
            a = self.sigmoid(z)
            self.cache.append({'a': a, 'z': z})
        z_out = a @ self.out['W'] + self.out['b']
        a_out = self.softmax(z_out)
        self.cache.append({'a': a_out, 'z': z_out})
        return a_out

    def backward(self, X, Y, out):
        m = X.shape[0]
        dz = (out - Y) / m

        # output layer update
        a_prev = self.cache[-2]['a']
        dW = a_prev.T @ dz
        db = np.sum(dz, axis=0, keepdims=True)
        self.out['W'] -= self.lr * dW
        self.out['b'] -= self.lr * db
        da = dz @ self.out['W'].T

        # hidden layers
        for i in reversed(range(len(self.layers))):
            a_curr = self.cache[i+1]['a']
            a_prev = self.cache[i]['a']
            dz = da * self.sigmoid_deriv(a_curr)
            dW = a_prev.T @ dz
            db = np.sum(dz, axis=0, keepdims=True)
            self.layers[i]['W'] -= self.lr * dW
            self.layers[i]['b'] -= self.lr * db
            da = dz @ self.layers[i]['W'].T

    def loss(self, Y, out):
        return -np.mean(np.sum(Y * np.log(out + 1e-9), axis=1))

    def predict(self, X):
        probs = self.forward(X)
        return np.argmax(probs, axis=1)




In [6]:
class ANN_r:
    def __init__(self, input_dim, hidden_layers, output_dim, lr=0.1):
        self.lr = lr
        self.layers = []
        prev = input_dim
        for h in hidden_layers:
            self.layers.append({
                "W": np.random.randn(prev, h) * np.sqrt(2.0 / prev),  # He init for ReLU
                "b": np.zeros((1, h))
            })
            prev = h
        self.out = {
            "W": np.random.randn(prev, output_dim) * 0.01,
            "b": np.zeros((1, output_dim))
        }

    # ReLU activation and derivative
    def relu(self, z):
        return np.maximum(0, z)

    def relu_deriv(self, z):
        return (z > 0).astype(float)

    def softmax(self, z):
        z -= np.max(z, axis=1, keepdims=True)
        exp = np.exp(z)
        return exp / np.sum(exp, axis=1, keepdims=True)

    def forward(self, X):
        self.cache = [{'a': X}]
        a = X
        for layer in self.layers:
            z = a @ layer['W'] + layer['b']
            a = self.relu(z)
            self.cache.append({'a': a, 'z': z})
        z_out = a @ self.out['W'] + self.out['b']
        a_out = self.softmax(z_out)
        self.cache.append({'a': a_out, 'z': z_out})
        return a_out

    def backward(self, X, Y, out):
        m = X.shape[0]
        dz = (out - Y) / m

        # output layer update
        a_prev = self.cache[-2]['a']
        dW = a_prev.T @ dz
        db = np.sum(dz, axis=0, keepdims=True)
        self.out['W'] -= self.lr * dW
        self.out['b'] -= self.lr * db
        da = dz @ self.out['W'].T

        # hidden layers backward
        for i in reversed(range(len(self.layers))):
            z_curr = self.cache[i+1]['z']
            a_prev = self.cache[i]['a']
            dz = da * self.relu_deriv(z_curr)
            dW = a_prev.T @ dz
            db = np.sum(dz, axis=0, keepdims=True)
            self.layers[i]['W'] -= self.lr * dW
            self.layers[i]['b'] -= self.lr * db
            da = dz @ self.layers[i]['W'].T

    def loss(self, Y, out):
        return -np.mean(np.sum(Y * np.log(out + 1e-9), axis=1))

    def predict(self, X):
        probs = self.forward(X)
        return np.argmax(probs, axis=1)


In [7]:
def train_model(X_train, Y_train, X_test, Y_test, layers, name, epochs=10, lr=0.1):
    model = ANN(3072, layers, 10, lr)
    best_acc = 0
    for epoch in range(epochs):
        out = model.forward(X_train)
        loss = model.loss(Y_train, out)
        model.backward(X_train, Y_train, out)
        preds = model.predict(X_test)
        acc = accuracy_score(np.argmax(Y_test, axis=1), preds)
        print(f"{name} | Epoch {epoch+1}/{epochs} | Loss={loss:.4f} | Acc={acc:.4f}")
        if acc > best_acc:
            best_acc = acc
            np.savez(f"{name}_best_weights.npz",
                     layers=model.layers, out=model.out)
    print(f"Best accuracy for {name}: {best_acc:.4f}")
    return model



In [8]:
def train_model_r(X_train, Y_train, X_test, Y_test, layers, name, epochs=10, lr=0.1):
    model = ANN_r(3072, layers, 10, lr)
    best_acc = 0
    for epoch in range(epochs):
        out = model.forward(X_train)
        loss = model.loss(Y_train, out)
        model.backward(X_train, Y_train, out)
        preds = model.predict(X_test)
        acc = accuracy_score(np.argmax(Y_test, axis=1), preds)
        print(f"{name} | Epoch {epoch+1}/{epochs} | Loss={loss:.4f} | Acc={acc:.4f}")
        if acc > best_acc:
            best_acc = acc
            np.savez(f"{name}_best_weights.npz",
                     layers=model.layers, out=model.out)
    print(f"Best accuracy for {name}: {best_acc:.4f}")
    return model



In [9]:
X_train, y_train = load_cifar10("cifar")
X_test, y_test = load_cifar10_test("cifar")

enc = OneHotEncoder(sparse_output=False)
Y_train = enc.fit_transform(y_train)
Y_test = enc.transform(y_test)



In [11]:
model=train_model(X_train, Y_train, X_test, Y_test, [100], "model_1layer", epochs=0, lr=0.05)

model_1layer | Epoch 1/40 | Loss=2.3042 | Acc=0.1140
model_1layer | Epoch 2/40 | Loss=2.3039 | Acc=0.1154
model_1layer | Epoch 3/40 | Loss=2.3036 | Acc=0.1165
model_1layer | Epoch 4/40 | Loss=2.3034 | Acc=0.1171
model_1layer | Epoch 5/40 | Loss=2.3032 | Acc=0.1186
model_1layer | Epoch 6/40 | Loss=2.3030 | Acc=0.1189
model_1layer | Epoch 7/40 | Loss=2.3029 | Acc=0.1188
model_1layer | Epoch 8/40 | Loss=2.3027 | Acc=0.1229
model_1layer | Epoch 9/40 | Loss=2.3026 | Acc=0.1263
model_1layer | Epoch 10/40 | Loss=2.3025 | Acc=0.1312
model_1layer | Epoch 11/40 | Loss=2.3024 | Acc=0.1343
model_1layer | Epoch 12/40 | Loss=2.3023 | Acc=0.1345
model_1layer | Epoch 13/40 | Loss=2.3022 | Acc=0.1390
model_1layer | Epoch 14/40 | Loss=2.3021 | Acc=0.1394
model_1layer | Epoch 15/40 | Loss=2.3021 | Acc=0.1427
model_1layer | Epoch 16/40 | Loss=2.3020 | Acc=0.1434
model_1layer | Epoch 17/40 | Loss=2.3019 | Acc=0.1425
model_1layer | Epoch 18/40 | Loss=2.3018 | Acc=0.1440
model_1layer | Epoch 19/40 | Loss=2.3

In [13]:
model2=train_model_r(X_train, Y_train, X_test, Y_test, [100], "model_1layer", epochs=40, lr=0.1)

model_1layer | Epoch 1/40 | Loss=2.3081 | Acc=0.1132
model_1layer | Epoch 2/40 | Loss=2.3012 | Acc=0.1239
model_1layer | Epoch 3/40 | Loss=2.2963 | Acc=0.1182
model_1layer | Epoch 4/40 | Loss=2.2922 | Acc=0.1183
model_1layer | Epoch 5/40 | Loss=2.2886 | Acc=0.1191
model_1layer | Epoch 6/40 | Loss=2.2851 | Acc=0.1234
model_1layer | Epoch 7/40 | Loss=2.2815 | Acc=0.1269
model_1layer | Epoch 8/40 | Loss=2.2778 | Acc=0.1330
model_1layer | Epoch 9/40 | Loss=2.2739 | Acc=0.1386
model_1layer | Epoch 10/40 | Loss=2.2697 | Acc=0.1462
model_1layer | Epoch 11/40 | Loss=2.2652 | Acc=0.1566
model_1layer | Epoch 12/40 | Loss=2.2604 | Acc=0.1667
model_1layer | Epoch 13/40 | Loss=2.2552 | Acc=0.1734
model_1layer | Epoch 14/40 | Loss=2.2497 | Acc=0.1795
model_1layer | Epoch 15/40 | Loss=2.2439 | Acc=0.1867
model_1layer | Epoch 16/40 | Loss=2.2378 | Acc=0.1913
model_1layer | Epoch 17/40 | Loss=2.2313 | Acc=0.1978
model_1layer | Epoch 18/40 | Loss=2.2246 | Acc=0.2022
model_1layer | Epoch 19/40 | Loss=2.2

In [14]:
model3=train_model_r(X_train, Y_train, X_test, Y_test, [100], "model_1layer", epochs=40, lr=0.3)

model_1layer | Epoch 1/40 | Loss=2.3042 | Acc=0.1216
model_1layer | Epoch 2/40 | Loss=2.2958 | Acc=0.1169
model_1layer | Epoch 3/40 | Loss=2.2882 | Acc=0.1265
model_1layer | Epoch 4/40 | Loss=2.2797 | Acc=0.1503
model_1layer | Epoch 5/40 | Loss=2.2691 | Acc=0.1727
model_1layer | Epoch 6/40 | Loss=2.2558 | Acc=0.1905
model_1layer | Epoch 7/40 | Loss=2.2389 | Acc=0.2057
model_1layer | Epoch 8/40 | Loss=2.2185 | Acc=0.2191
model_1layer | Epoch 9/40 | Loss=2.1953 | Acc=0.2268
model_1layer | Epoch 10/40 | Loss=2.1709 | Acc=0.2413
model_1layer | Epoch 11/40 | Loss=2.1477 | Acc=0.2205
model_1layer | Epoch 12/40 | Loss=2.1428 | Acc=0.1115
model_1layer | Epoch 13/40 | Loss=2.5801 | Acc=0.1015
model_1layer | Epoch 14/40 | Loss=6.1075 | Acc=0.1000
model_1layer | Epoch 15/40 | Loss=17.0865 | Acc=0.1000
model_1layer | Epoch 16/40 | Loss=18.6509 | Acc=0.1000
model_1layer | Epoch 17/40 | Loss=18.6509 | Acc=0.1000
model_1layer | Epoch 18/40 | Loss=18.6509 | Acc=0.1000
model_1layer | Epoch 19/40 | Loss

In [15]:
def diagnose_model(model, X_train, Y_train, X_val, Y_val):
    # Forward passes
    out_train = model.forward(X_train)
    out_val = model.forward(X_val)

    # Compute losses
    train_loss = model.loss(Y_train, out_train)
    val_loss = model.loss(Y_val, out_val)

    # Compute accuracies
    train_pred = np.argmax(out_train, axis=1)
    val_pred = np.argmax(out_val, axis=1)
    y_train_true = np.argmax(Y_train, axis=1)
    y_val_true = np.argmax(Y_val, axis=1)

    train_acc = np.mean(train_pred == y_train_true)
    val_acc = np.mean(val_pred == y_val_true)

    print(f"Train loss: {train_loss:.4f}, Val loss: {val_loss:.4f}")
    print(f"Train acc: {train_acc:.4f}, Val acc: {val_acc:.4f}")

    # Diagnose
    if train_acc > 0.9 and val_acc < 0.7:
        print("Diagnosis: Overfitting")
    elif train_acc < 0.7 and val_acc < 0.7:
        print("Diagnosis: Underfitting")
    else:
        print("Diagnosis: Good fit")



In [16]:

diagnose_model(model, X_train, Y_train, X_test, Y_test)

Train loss: 2.0740, Val loss: 2.0745
Train acc: 0.2572, Val acc: 0.2559
Diagnosis: Underfitting


In [17]:
diagnose_model(model2, X_train, Y_train, X_test, Y_test)

Train loss: 2.0740, Val loss: 2.0745
Train acc: 0.2572, Val acc: 0.2559
Diagnosis: Underfitting


In [18]:
diagnose_model(model3, X_train, Y_train, X_test, Y_test)

Train loss: 18.6509, Val loss: 18.6509
Train acc: 0.1000, Val acc: 0.1000
Diagnosis: Underfitting
